In [1]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from torchvision.utils import save_image
device = torch.device("mps")

In [2]:
content_img = '/Users/aadityajoshi/Downloads/content.png'

In [3]:
style_img = '/Users/aadityajoshi/Downloads/style.png'

### define both losses

### Content loss

In [4]:
def calc_content_loss(gen_feat,orig_feat):
    #calculating the content loss of each layer by calculating the MSE between the content and generated features and adding it to content loss
    content_l=torch.mean((gen_feat-orig_feat)**2)
    return content_l

In [5]:
def calc_style_loss(gen,style):
    #Calculating the gram matrix for the style and the generated image
    batch_size,channel,height,width=gen.shape

    G=torch.mm(gen.view(channel,height*width),gen.view(channel,height*width).t())
    A=torch.mm(style.view(channel,height*width),style.view(channel,height*width).t())
        
    #Calcultating the style loss of each layer by calculating the MSE between the gram matrix of the style image and the generated image and adding it to style loss
    style_l=torch.mean((G-A)**2)
    return style_l

In [6]:
def calculate_loss(gen_features, orig_feautes, style_featues):
    style_loss=content_loss=0
    for gen,cont,style in zip(gen_features,orig_feautes,style_featues):
        #extracting the dimensions from the generated image
        content_loss+=calc_content_loss(gen,cont)
        style_loss+=calc_style_loss(gen,style)
    
    #calculating the total loss of e th epoch
    total_loss=alpha*content_loss + beta*style_loss 
    return total_loss

In [7]:
img_size = 496
loader = transforms.Compose([
        transforms.Resize(img_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])])

In [8]:
def image_loader(path):
    image=Image.open(path)
    #defining the image transformation steps to be performed before feeding them to the model
    loader=transforms.Compose([transforms.Resize((512,512)), transforms.ToTensor()])
    #The preprocessing steps involves resizing the image and then converting it to a tensor
    image=loader(image).unsqueeze(0)
    return image.to(device,torch.float)

#Loading the original and the style image
original_image=image_loader('/Users/aadityajoshi/Downloads/content.png')
original_image = original_image[:, :3, :, :] 
style_image=image_loader('/Users/aadityajoshi/Downloads/style.png')
style_image = style_image[:, :3, :, :]

#Creating the generated image from the original image
generated_image=original_image.clone().requires_grad_(True)

In [9]:
class VGG(nn.Module):
    def __init__(self):
        super(VGG,self).__init__()
        self.req_features= ['0','5','10','19','28'] 
        #Since we need only the 5 layers in the model so we will be dropping all the rest layers from the features of the model
        self.model= models.vgg19(weights='VGG19_Weights.DEFAULT').features[:29]#model will contain the first 29 layers
    #x holds the input tensor(image) that will be feeded to each layer
    def forward(self,x):
        #initialize an array that wil hold the activations from the chosen layers
        features=[]
        #Iterate over all the layers of the mode
        for layer_num,layer in enumerate(self.model):
            #activation of the layer will stored in x
            x=layer(x)
            #appending the activation of the selected layers and return the feature array
            if (str(layer_num) in self.req_features):
                features.append(x)
                
        return features

In [10]:
model=VGG().to(device).eval() 

#initialize the paramerters required for fitting the model
epoch=7000
lr=0.004
alpha=8
beta=70

#using adam optimizer and it will update the generated image not the model parameter 
optimizer=torch.optim.Adam([generated_image],lr=lr)

In [11]:
for e in range (epoch):
    #extracting the features of generated, content and the original required for calculating the loss
    gen_features=model(generated_image)
    orig_feautes=model(original_image)
    style_featues=model(style_image)
    
    #iterating over the activation of each layer and calculate the loss and add it to the content and the style loss
    total_loss=calculate_loss(gen_features, orig_feautes, style_featues)
    #optimize the pixel values of the generated image and backpropagate the loss
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    #print the image and save it after each 100 epoch
    if(e/100):
        print(total_loss)
        
        save_image(generated_image,"gen.png")    

tensor(1.1939e+11, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.1588e+11, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.1197e+11, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.0772e+11, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.0329e+11, device='mps:0', grad_fn=<AddBackward0>)
tensor(9.8889e+10, device='mps:0', grad_fn=<AddBackward0>)
tensor(9.4591e+10, device='mps:0', grad_fn=<AddBackward0>)
tensor(9.0382e+10, device='mps:0', grad_fn=<AddBackward0>)
tensor(8.6244e+10, device='mps:0', grad_fn=<AddBackward0>)
tensor(8.2170e+10, device='mps:0', grad_fn=<AddBackward0>)
tensor(7.8178e+10, device='mps:0', grad_fn=<AddBackward0>)
tensor(7.4288e+10, device='mps:0', grad_fn=<AddBackward0>)
tensor(7.0530e+10, device='mps:0', grad_fn=<AddBackward0>)
tensor(6.6943e+10, device='mps:0', grad_fn=<AddBackward0>)
tensor(6.3557e+10, device='mps:0', grad_fn=<AddBackward0>)
tensor(6.0388e+10, device='mps:0', grad_fn=<AddBackward0>)
tensor(5.7440e+10, device='mps:0', grad_fn=<AddBackward0

tensor(1.0551e+10, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.0459e+10, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.0368e+10, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.0277e+10, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.0186e+10, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.0095e+10, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.0005e+10, device='mps:0', grad_fn=<AddBackward0>)
tensor(9.9151e+09, device='mps:0', grad_fn=<AddBackward0>)
tensor(9.8255e+09, device='mps:0', grad_fn=<AddBackward0>)
tensor(9.7363e+09, device='mps:0', grad_fn=<AddBackward0>)
tensor(9.6475e+09, device='mps:0', grad_fn=<AddBackward0>)
tensor(9.5591e+09, device='mps:0', grad_fn=<AddBackward0>)
tensor(9.4710e+09, device='mps:0', grad_fn=<AddBackward0>)
tensor(9.3833e+09, device='mps:0', grad_fn=<AddBackward0>)
tensor(9.2961e+09, device='mps:0', grad_fn=<AddBackward0>)
tensor(9.2092e+09, device='mps:0', grad_fn=<AddBackward0>)
tensor(9.1227e+09, device='mps:0', grad_fn=<AddBackward0

tensor(2.8379e+09, device='mps:0', grad_fn=<AddBackward0>)
tensor(2.8141e+09, device='mps:0', grad_fn=<AddBackward0>)
tensor(2.7906e+09, device='mps:0', grad_fn=<AddBackward0>)
tensor(2.7673e+09, device='mps:0', grad_fn=<AddBackward0>)
tensor(2.7442e+09, device='mps:0', grad_fn=<AddBackward0>)
tensor(2.7214e+09, device='mps:0', grad_fn=<AddBackward0>)
tensor(2.6987e+09, device='mps:0', grad_fn=<AddBackward0>)
tensor(2.6763e+09, device='mps:0', grad_fn=<AddBackward0>)
tensor(2.6540e+09, device='mps:0', grad_fn=<AddBackward0>)
tensor(2.6320e+09, device='mps:0', grad_fn=<AddBackward0>)
tensor(2.6102e+09, device='mps:0', grad_fn=<AddBackward0>)
tensor(2.5885e+09, device='mps:0', grad_fn=<AddBackward0>)
tensor(2.5671e+09, device='mps:0', grad_fn=<AddBackward0>)
tensor(2.5458e+09, device='mps:0', grad_fn=<AddBackward0>)
tensor(2.5248e+09, device='mps:0', grad_fn=<AddBackward0>)
tensor(2.5039e+09, device='mps:0', grad_fn=<AddBackward0>)
tensor(2.4833e+09, device='mps:0', grad_fn=<AddBackward0

tensor(1.0095e+09, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.0038e+09, device='mps:0', grad_fn=<AddBackward0>)
tensor(9.9822e+08, device='mps:0', grad_fn=<AddBackward0>)
tensor(9.9268e+08, device='mps:0', grad_fn=<AddBackward0>)
tensor(9.8721e+08, device='mps:0', grad_fn=<AddBackward0>)
tensor(9.8181e+08, device='mps:0', grad_fn=<AddBackward0>)
tensor(9.7647e+08, device='mps:0', grad_fn=<AddBackward0>)
tensor(9.7120e+08, device='mps:0', grad_fn=<AddBackward0>)
tensor(9.6598e+08, device='mps:0', grad_fn=<AddBackward0>)
tensor(9.6083e+08, device='mps:0', grad_fn=<AddBackward0>)
tensor(9.5574e+08, device='mps:0', grad_fn=<AddBackward0>)
tensor(9.5071e+08, device='mps:0', grad_fn=<AddBackward0>)
tensor(9.4574e+08, device='mps:0', grad_fn=<AddBackward0>)
tensor(9.4083e+08, device='mps:0', grad_fn=<AddBackward0>)
tensor(9.3598e+08, device='mps:0', grad_fn=<AddBackward0>)
tensor(9.3119e+08, device='mps:0', grad_fn=<AddBackward0>)
tensor(9.2645e+08, device='mps:0', grad_fn=<AddBackward0

tensor(6.1003e+08, device='mps:0', grad_fn=<AddBackward0>)
tensor(6.0867e+08, device='mps:0', grad_fn=<AddBackward0>)
tensor(6.0733e+08, device='mps:0', grad_fn=<AddBackward0>)
tensor(6.0599e+08, device='mps:0', grad_fn=<AddBackward0>)
tensor(6.0466e+08, device='mps:0', grad_fn=<AddBackward0>)
tensor(6.0334e+08, device='mps:0', grad_fn=<AddBackward0>)
tensor(6.0203e+08, device='mps:0', grad_fn=<AddBackward0>)
tensor(6.0072e+08, device='mps:0', grad_fn=<AddBackward0>)
tensor(5.9943e+08, device='mps:0', grad_fn=<AddBackward0>)
tensor(5.9814e+08, device='mps:0', grad_fn=<AddBackward0>)
tensor(5.9686e+08, device='mps:0', grad_fn=<AddBackward0>)
tensor(5.9560e+08, device='mps:0', grad_fn=<AddBackward0>)
tensor(5.9433e+08, device='mps:0', grad_fn=<AddBackward0>)
tensor(5.9308e+08, device='mps:0', grad_fn=<AddBackward0>)
tensor(5.9184e+08, device='mps:0', grad_fn=<AddBackward0>)
tensor(5.9060e+08, device='mps:0', grad_fn=<AddBackward0>)
tensor(5.8937e+08, device='mps:0', grad_fn=<AddBackward0

...

tensor(84831896., device='mps:0', grad_fn=<AddBackward0>)
tensor(84786568., device='mps:0', grad_fn=<AddBackward0>)
tensor(84727704., device='mps:0', grad_fn=<AddBackward0>)
tensor(84654424., device='mps:0', grad_fn=<AddBackward0>)
tensor(84581960., device='mps:0', grad_fn=<AddBackward0>)
tensor(84521472., device='mps:0', grad_fn=<AddBackward0>)
tensor(84477104., device='mps:0', grad_fn=<AddBackward0>)
tensor(84444320., device='mps:0', grad_fn=<AddBackward0>)
tensor(84417808., device='mps:0', grad_fn=<AddBackward0>)
tensor(84400008., device='mps:0', grad_fn=<AddBackward0>)
tensor(84386976., device='mps:0', grad_fn=<AddBackward0>)
tensor(84377600., device='mps:0', grad_fn=<AddBackward0>)
tensor(84349104., device='mps:0', grad_fn=<AddBackward0>)
tensor(84292392., device='mps:0', grad_fn=<AddBackward0>)
tensor(84196984., device='mps:0', grad_fn=<AddBackward0>)
tensor(84095800., device='mps:0', grad_fn=<AddBackward0>)
tensor(84024880., device='mps:0', grad_fn=<AddBackward0>)
tensor(8399484

tensor(78688152., device='mps:0', grad_fn=<AddBackward0>)
tensor(78693288., device='mps:0', grad_fn=<AddBackward0>)
tensor(78677448., device='mps:0', grad_fn=<AddBackward0>)
tensor(78629592., device='mps:0', grad_fn=<AddBackward0>)
tensor(78525952., device='mps:0', grad_fn=<AddBackward0>)
tensor(78405312., device='mps:0', grad_fn=<AddBackward0>)
tensor(78314304., device='mps:0', grad_fn=<AddBackward0>)
tensor(78284304., device='mps:0', grad_fn=<AddBackward0>)
tensor(78290720., device='mps:0', grad_fn=<AddBackward0>)
tensor(78294408., device='mps:0', grad_fn=<AddBackward0>)
tensor(78281136., device='mps:0', grad_fn=<AddBackward0>)
tensor(78231752., device='mps:0', grad_fn=<AddBackward0>)
tensor(78155304., device='mps:0', grad_fn=<AddBackward0>)
tensor(78061424., device='mps:0', grad_fn=<AddBackward0>)
tensor(77983432., device='mps:0', grad_fn=<AddBackward0>)
tensor(77936720., device='mps:0', grad_fn=<AddBackward0>)
tensor(77915560., device='mps:0', grad_fn=<AddBackward0>)
tensor(7790952

tensor(73055640., device='mps:0', grad_fn=<AddBackward0>)
tensor(72956456., device='mps:0', grad_fn=<AddBackward0>)
tensor(72901008., device='mps:0', grad_fn=<AddBackward0>)
tensor(72886776., device='mps:0', grad_fn=<AddBackward0>)
tensor(72893832., device='mps:0', grad_fn=<AddBackward0>)
tensor(72913416., device='mps:0', grad_fn=<AddBackward0>)
tensor(72939096., device='mps:0', grad_fn=<AddBackward0>)
tensor(72975040., device='mps:0', grad_fn=<AddBackward0>)
tensor(72990072., device='mps:0', grad_fn=<AddBackward0>)
tensor(72965416., device='mps:0', grad_fn=<AddBackward0>)
tensor(72851216., device='mps:0', grad_fn=<AddBackward0>)
tensor(72686600., device='mps:0', grad_fn=<AddBackward0>)
tensor(72551664., device='mps:0', grad_fn=<AddBackward0>)
tensor(72520368., device='mps:0', grad_fn=<AddBackward0>)
tensor(72560040., device='mps:0', grad_fn=<AddBackward0>)
tensor(72596656., device='mps:0', grad_fn=<AddBackward0>)
tensor(72594528., device='mps:0', grad_fn=<AddBackward0>)
tensor(7252732

tensor(68177520., device='mps:0', grad_fn=<AddBackward0>)
tensor(68187976., device='mps:0', grad_fn=<AddBackward0>)
tensor(68182824., device='mps:0', grad_fn=<AddBackward0>)
tensor(68117832., device='mps:0', grad_fn=<AddBackward0>)
tensor(68010688., device='mps:0', grad_fn=<AddBackward0>)
tensor(67874888., device='mps:0', grad_fn=<AddBackward0>)
tensor(67778656., device='mps:0', grad_fn=<AddBackward0>)
tensor(67750304., device='mps:0', grad_fn=<AddBackward0>)
tensor(67769264., device='mps:0', grad_fn=<AddBackward0>)
tensor(67805920., device='mps:0', grad_fn=<AddBackward0>)
tensor(67836584., device='mps:0', grad_fn=<AddBackward0>)
tensor(67864344., device='mps:0', grad_fn=<AddBackward0>)
tensor(67856904., device='mps:0', grad_fn=<AddBackward0>)
tensor(67813416., device='mps:0', grad_fn=<AddBackward0>)
tensor(67694872., device='mps:0', grad_fn=<AddBackward0>)
tensor(67553368., device='mps:0', grad_fn=<AddBackward0>)
tensor(67448024., device='mps:0', grad_fn=<AddBackward0>)
tensor(6742264

KeyboardInterrupt: 